### Experiement with how to reduct the docuemnt to be parsed. Number of tokens is too hight. Try to experiement with date.

In [109]:
import openai
import docx
import pandas as pd
FILE_NAME = "/Users/prashantgaurav/Projects/KaajAi/receipts/bank-statement-template-20.docx"

def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

In [110]:
import openai
import os
client = openai.Client(api_key=os.environ.get("OPENAI_API_KEY"))
text = getText(FILE_NAME)

In [111]:
doc = docx.Document(FILE_NAME)
text = ""
for table in doc.tables:
    for row in table.rows:
        for cell in row.cells:
            text+= cell.text

In [112]:
text

'CHECKING SUMMARY\nINSTANCES\nAMOUNTBeginning Balance$81,607.40Deposits and Additions10125,883.63Checks Paid2- 3,169.04DATEDESCRIPTIONAMOUNT07/02Deposit$17,120.0007/09Deposit24,610.0007/14Deposit11,424.0007/15Deposit1,349.0007/21Deposit5,000.0007/21Deposit3,120.0007/23Deposit33,138.0007/28Deposit18,114.0007/30Deposit6,908.6307/30Deposit5,100.00\nCHECK NUMBER\nDESCRIPTIONDATE PAID\nAMOUNTXXXX\t^07/14$1,471.99XXXX\t^07/081,697.05Total Checks Paid$3,169.04DATEDESCRIPTIONAMOUNT07/11Online Payment XXXXX To Vendor$8,928.0007/11Online Payment XXXXX To Vendor2,960.0007/25Online Payment XXXXX To Vendor250.0007/30ADP TX/Fincl Svc ADP2,887.68DATEAMOUNTDATEAMOUNT07/02$98,727.4007/21129,173.3607/0897,030.3507/23162,311.3607/09121,640.3507/25162,061.3607/11109,752.3507/28180,175.3607/14108,280.3607/30189,296.3107/16121,053.36'

In [113]:
%%time
prompt = """I'm going to give you the contents of a pdf of a bank statement. The text containes transaction records along with other things.
you job is to parse each transaction record and return the details like date, description, amount and type. The type column should contain either 
'credit' or 'debit'. Return these information in a tabular format. Do not return anything else just the table"""

docs = [{"role": "system", "content": prompt + '\n' + text}]
response = client.chat.completions.create(
                model="gpt-4",
                messages=docs
            )

resp = response.choices[0].message.content

CPU times: user 17.4 ms, sys: 12.8 ms, total: 30.2 ms
Wall time: 18.8 s


In [114]:
print(resp)

| Date       | Description                   | Amount    | Type   |
|------------|-------------------------------|-----------|--------|
| 07/02      | Deposit                       | $17,120.00  | Credit |
| 07/09      | Deposit                       | $24,610.00  | Credit |
| 07/14      | Deposit                       | $11,424.00  | Credit |
| 07/15      | Deposit                       | $1,349.00   | Credit |
| 07/21      | Deposit                       | $5,000.00   | Credit |
| 07/21      | Deposit                       | $3,120.00   | Credit |
| 07/23      | Deposit                       | $33,138.00  | Credit |
| 07/28      | Deposit                       | $18,114.00  | Credit |
| 07/30      | Deposit                       | $6,908.63   | Credit |
| 07/30      | Deposit                       | $5,100.00   | Credit |
| 07/14      | Check Paid                    | $1,471.99   | Debit  |
| 07/08      | Check Paid                    | $1,697.05   | Debit  |
| 07/11      | Online Pa

In [115]:
def parse_response_table(resp):
    lines = resp.strip().split("\n")
    table = [line.split('|') for line in lines]
    final_table = []
    for line in table:
        new_line = []
        for ele in line:
            ele = ele.replace('-','')
            if ele :
               new_line.append(ele.strip())
        if not new_line:
            continue
        final_table.append(new_line)
    print(final_table)
    df = pd.DataFrame(final_table[1:],columns=final_table[0])
    return df
df = parse_response_table(resp)

[['Date', 'Description', 'Amount', 'Type'], ['07/02', 'Deposit', '$17,120.00', 'Credit'], ['07/09', 'Deposit', '$24,610.00', 'Credit'], ['07/14', 'Deposit', '$11,424.00', 'Credit'], ['07/15', 'Deposit', '$1,349.00', 'Credit'], ['07/21', 'Deposit', '$5,000.00', 'Credit'], ['07/21', 'Deposit', '$3,120.00', 'Credit'], ['07/23', 'Deposit', '$33,138.00', 'Credit'], ['07/28', 'Deposit', '$18,114.00', 'Credit'], ['07/30', 'Deposit', '$6,908.63', 'Credit'], ['07/30', 'Deposit', '$5,100.00', 'Credit'], ['07/14', 'Check Paid', '$1,471.99', 'Debit'], ['07/08', 'Check Paid', '$1,697.05', 'Debit'], ['07/11', 'Online Payment To Vendor', '$8,928.00', 'Debit'], ['07/11', 'Online Payment To Vendor', '$2,960.00', 'Debit'], ['07/25', 'Online Payment To Vendor', '$250.00', 'Debit'], ['07/30', 'ADP TX/Fincl Svc ADP', '$2,887.68', 'Debit']]


In [116]:
df

,Date,Description,Amount,Type
0,07/02,Deposit,"$17,120.00",Credit
1,07/09,Deposit,"$24,610.00",Credit
2,07/14,Deposit,"$11,424.00",Credit
3,07/15,Deposit,"$1,349.00",Credit
4,07/21,Deposit,"$5,000.00",Credit
5,07/21,Deposit,"$3,120.00",Credit
6,07/23,Deposit,"$33,138.00",Credit
7,07/28,Deposit,"$18,114.00",Credit
8,07/30,Deposit,"$6,908.63",Credit
9,07/30,Deposit,"$5,100.00",Credit
